### 豆瓣评分的预测

在这个项目中，我们要预测一部电影的评分，这个问题实际上就是一个分类问题。给定的输入为一段文本，输出为具体的评分。 在这个项目中，我们需要做：
- 文本的预处理，如停用词的过滤，低频词的过滤，特殊符号的过滤等
- 文本转化成向量，将使用三种方式，分别为tf-idf, word2vec以及BERT向量。 
- 训练逻辑回归和朴素贝叶斯模型，并做交叉验证
- 评估模型的准确率

在具体标记为``TODO``的部分填写相应的代码。 

In [1]:
#导入数据处理的基础包
import numpy as np
import pandas as pd

#导入用于计数的包
from collections import Counter

#导入tf-idf相关的包
from sklearn.feature_extraction.text import TfidfTransformer    
from sklearn.feature_extraction.text import CountVectorizer

#导入模型评估的包
from sklearn import metrics

#导入与word2vec相关的包
from gensim.models import KeyedVectors

#导入与bert embedding相关的包，关于mxnet包下载的注意事项参考实验手册
from bert_embedding import BertEmbedding
import mxnet

#包tqdm是用来对可迭代对象执行时生成一个进度条用以监视程序运行过程
from tqdm import tqdm

#导入其他一些功能包
import requests
import os

import re

# Ignore the warnings
import warnings
warnings.filterwarnings('always')
warnings.filterwarnings('ignore')

### 1. 读取数据并做文本的处理
你需要完成以下几步操作：
- 去掉无用的字符如！&，可自行定义
- 中文分词
- 去掉低频词

In [2]:
#读取数据
data = pd.read_csv('./data/DMSC.csv')
#观察数据格式
data.head(50)

,ID,Movie_Name_EN,Movie_Name_CN,Crawl_Date,Number,Username,Date,Star,Comment,Like
0,0,Avengers Age of Ultron,复仇者联盟2,2017-01-22,1,然潘,2015-05-13,3,连奥创都知道整容要去韩国。,2404
1,10,Avengers Age of Ultron,复仇者联盟2,2017-01-22,11,影志,2015-04-30,4,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,381
2,20,Avengers Age of Ultron,复仇者联盟2,2017-01-22,21,随时流感,2015-04-28,2,奥创弱爆了弱爆了弱爆了啊！！！！！！,120
3,30,Avengers Age of Ultron,复仇者联盟2,2017-01-22,31,乌鸦火堂,2015-05-08,4,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,30
4,40,Avengers Age of Ultron,复仇者联盟2,2017-01-22,41,办公室甜心,2015-05-10,5,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,16
5,50,Avengers Age of Ultron,复仇者联盟2,2017-01-22,51,羚羊的灵魂,2015-05-12,5,绝逼不质疑尾灯的导演和编剧水平,15
6,60,Avengers Age of Ultron,复仇者联盟2,2017-01-22,61,旺财,2015-05-02,2,avengers1睡着1次 avengers2睡着两次。。。,26
7,70,Avengers Age of Ultron,复仇者联盟2,2017-01-22,72,小黄,2015-05-12,1,谁再喊我看这种电影我和谁急！实在是接受无能。。。,9
8,80,Avengers Age of Ultron,复仇者联盟2,2017-01-22,82,旧书君,2015-05-12,5,超愉悦以及超满足。在历经了第一阶段比漫画更普世的设定融合之后，发展到#AoU#居然出现了不...,5
9,90,Avengers Age of Ultron,复仇者联盟2,2017-01-22,93,赵大宝,2015-05-14,3,观影过程中，耳边一直有一种突突突突突的声音，我还感慨电影为了让奥创给观众带来紧张感，声音上...,14


In [3]:
#可参照 Titanic_prediction 示例进行更多数据探索分析
data['Movie_Name_CN'].value_counts()

疯狂动物城     13751
大圣归来      13339
后会无期      12020
寻龙诀       11369
你的名字      11326
夏洛特烦恼     10916
釜山行       10287
爱乐之城       9662
西游伏妖篇      9145
小时代1       8891
泰囧         8568
大鱼海棠       8369
长城         8317
西游降魔篇      7996
复仇者联盟      7828
美人鱼        7388
七月与安生      6836
美国队长3      6441
变形金刚4      5875
复仇者联盟2     5416
十二生肖       4623
九层妖塔       4437
小时代3       4115
左耳         3980
湄公河行动      3509
栀子花开       3048
何以笙箫默      2680
钢铁侠1       2374
Name: Movie_Name_CN, dtype: int64

In [4]:
#输出数据的一些相关信息
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 212506 entries, 0 to 212505
Data columns (total 10 columns):
 #   Column         Non-Null Count   Dtype 
---  ------         --------------   ----- 
 0   ID             212506 non-null  int64 
 1   Movie_Name_EN  212506 non-null  object
 2   Movie_Name_CN  212506 non-null  object
 3   Crawl_Date     212506 non-null  object
 4   Number         212506 non-null  int64 
 5   Username       212496 non-null  object
 6   Date           212506 non-null  object
 7   Star           212506 non-null  int64 
 8   Comment        212506 non-null  object
 9   Like           212506 non-null  int64 
dtypes: int64(4), object(6)
memory usage: 16.2+ MB


In [5]:
#只保留数据中我们需要的两列：Comment列和Star列
data = data[['Comment','Star']]
#观察新的数据的格式
data.head()

,Comment,Star
0,连奥创都知道整容要去韩国。,3
1,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,4
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,2
3,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,4
4,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,5


In [6]:
# 这里的star代表具体的评分。但在这个项目中，我们要预测的是正面还是负面。我们把评分为1和2的看作是负面，把评分为3，4，5的作为正面
data['Star']=(data.Star/3).astype(int)
data.head()

,Comment,Star
0,连奥创都知道整容要去韩国。,1
1,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,1
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,0
3,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,1
4,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,1


In [7]:
data['Star'].value_counts()

1    175781
0     36725
Name: Star, dtype: int64

In [8]:
# 只选 1k 个样本测试代码
data = data.iloc[0:1000, ]

In [9]:
data['Star'].value_counts()

1    838
0    162
Name: Star, dtype: int64

#### 任务1： 去掉一些无用的字符

In [10]:
# TODO1: 去掉一些无用的字符，自行定一个字符几何，并从文本中去掉
#    your to do 

def pre_process(input_str):
    # input_str = re.sub('[0-9]+', 'DIG', input_str)
    # 去除标点符号
    # input_str = re.sub(r"[{}]+".format(punc), " ", input_str)
    
    input_str = re.sub(
        "[0-9a-zA-Z\-\s+\.\!\/_,$%^*\(\)\+(+\"\')]+|[+——！，。？、~@#￥%……&*（）<>\[\]:：★◆【】《》;；=?？]+", " ", input_str)
    # 其他非中文字符
    input_str = re.sub(r"[^\u4e00-\u9fff]", " ", input_str)
    return input_str.strip()


# 正则去除标点符号
data['comment_clean'] = data['Comment'].apply(pre_process)

In [11]:
data.head(30)

,Comment,Star,comment_clean
0,连奥创都知道整容要去韩国。,1,连奥创都知道整容要去韩国
1,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,1,一个没有黑暗面的人不值得信任 第二部剥去冗长的铺垫 开场即高潮 一直到结束 会有人觉得只...
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,0,奥创弱爆了弱爆了弱爆了啊
3,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,1,与第一集不同 承上启下 阴郁严肃 但也不会不好看啊 除非本来就不喜欢漫威电影 场面更加宏大 ...
4,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,1,看毕 我激动地对友人说 等等奥创要来毁灭台北怎么办厚 她拍了拍我肩膀 没事 反正你买了两份旅...
5,绝逼不质疑尾灯的导演和编剧水平,1,绝逼不质疑尾灯的导演和编剧水平
6,avengers1睡着1次 avengers2睡着两次。。。,0,睡着 次 睡着两次
7,谁再喊我看这种电影我和谁急！实在是接受无能。。。,0,谁再喊我看这种电影我和谁急 实在是接受无能
8,超愉悦以及超满足。在历经了第一阶段比漫画更普世的设定融合之后，发展到#AoU#居然出现了不...,1,超愉悦以及超满足 在历经了第一阶段比漫画更普世的设定融合之后 发展到 居然出现了不少传统...
9,观影过程中，耳边一直有一种突突突突突的声音，我还感慨电影为了让奥创给观众带来紧张感，声音上...,1,观影过程中 耳边一直有一种突突突突突的声音 我还感慨电影为了让奥创给观众带来紧张感 声音上真...


#### 任务2：使用结巴分词对文本做分词

In [12]:
# TODO2: 导入中文分词包jieba, 并用jieba对原始文本做分词
import jieba
def comment_cut(content):
    # TODO: 使用结巴完成对每一个comment的分词
    # 分词并过滤空字符串
    return ' '.join([w for w in jieba.lcut(content.strip()) if len(w) > 0])

# 输出进度条
tqdm.pandas(desc='apply')
data['comment_processed'] = data['comment_clean'].progress_apply(comment_cut)

apply:   0%|          | 0/1000 [00:00<?, ?it/s]Building prefix dict from the default dictionary ...
Loading model from cache /var/folders/t4/p2t85myn6nd9pvb3b22d82jr0000gp/T/jieba.cache
Loading model cost 0.667 seconds.
Prefix dict has been built successfully.
apply: 100%|██████████| 1000/1000 [00:00<00:00, 1177.67it/s]


In [13]:
# 观察新的数据的格式
data.head()

,Comment,Star,comment_clean,comment_processed
0,连奥创都知道整容要去韩国。,1,连奥创都知道整容要去韩国,连 奥创 都 知道 整容 要 去 韩国
1,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,1,一个没有黑暗面的人不值得信任 第二部剥去冗长的铺垫 开场即高潮 一直到结束 会有人觉得只...,一个 没有 黑暗面 的 人 不 值得 信任 第二部 剥去 冗长 的 铺垫 开...
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,0,奥创弱爆了弱爆了弱爆了啊,奥创 弱 爆 了 弱 爆 了 弱 爆 了 啊
3,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,1,与第一集不同 承上启下 阴郁严肃 但也不会不好看啊 除非本来就不喜欢漫威电影 场面更加宏大 ...,与 第一集 不同 承上启下 阴郁 严肃 但 也 不会 不 好看 啊 除非 本...
4,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,1,看毕 我激动地对友人说 等等奥创要来毁灭台北怎么办厚 她拍了拍我肩膀 没事 反正你买了两份旅...,看毕 我 激动 地 对 友人 说 等等 奥创 要 来 毁灭 台北 怎么办 厚 她...


#### 任务3：设定停用词并去掉停用词

In [14]:
# TODO3: 设定停用词并从文本中去掉停用词

# 下载中文停用词表至data/stopWord.json中，下载地址:https://github.com/goto456/stopwords/
if not os.path.exists('./data/stopWord.json'):
    stopWord = requests.get("https://raw.githubusercontent.com/goto456/stopwords/master/cn_stopwords.txt")
    with open("./data/stopWord.json", "wb") as f:
         f.write(stopWord.content)

# 读取下载的停用词表，并保存在列表中
with open("./data/stopWord.json","r") as f:
    stopWords = f.read().split("\n")  
    
    
# 去除停用词
def rm_stop_word(input_str):
    # your code, remove stop words
    # TODO
    return [w for w in input_str.split() if w not in stopWords]

#这行代码中.progress_apply()函数的作用等同于.apply()函数的作用，只是写成.progress_apply()函数才能被tqdm包监控从而输出进度条。
data['comment_processed'] = data['comment_processed'].progress_apply(rm_stop_word)

apply: 100%|██████████| 1000/1000 [00:00<00:00, 5737.77it/s]


In [15]:
# 观察新的数据的格式
data.head()

,Comment,Star,comment_clean,comment_processed
0,连奥创都知道整容要去韩国。,1,连奥创都知道整容要去韩国,"[奥创, 知道, 整容, 韩国]"
1,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,1,一个没有黑暗面的人不值得信任 第二部剥去冗长的铺垫 开场即高潮 一直到结束 会有人觉得只...,"[一个, 没有, 黑暗面, 值得, 信任, 第二部, 剥去, 冗长, 铺垫, 开场, 高潮,..."
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,0,奥创弱爆了弱爆了弱爆了啊,"[奥创, 弱, 爆, 弱, 爆, 弱, 爆]"
3,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,1,与第一集不同 承上启下 阴郁严肃 但也不会不好看啊 除非本来就不喜欢漫威电影 场面更加宏大 ...,"[第一集, 不同, 承上启下, 阴郁, 严肃, 不会, 好看, 本来, 喜欢, 漫威, 电影..."
4,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,1,看毕 我激动地对友人说 等等奥创要来毁灭台北怎么办厚 她拍了拍我肩膀 没事 反正你买了两份旅...,"[看毕, 激动, 友人, 说, 奥创, 毁灭, 台北, 厚, 拍了拍, 肩膀, 没事, 反正..."


#### 任务4：去掉低频词，出现次数少于10次的词去掉

In [16]:
list(data['comment_processed'].values)

[['奥创', '知道', '整容', '韩国'],
 ['一个',
  '没有',
  '黑暗面',
  '值得',
  '信任',
  '第二部',
  '剥去',
  '冗长',
  '铺垫',
  '开场',
  '高潮',
  '一直',
  '结束',
  '会',
  '有人',
  '觉得',
  '剩',
  '动作',
  '特技',
  '蛮',
  '吃',
  '一套',
  '联盟',
  '阵营',
  '葫芦兄弟',
  '八仙过海',
  '式',
  '各显神通',
  '依然',
  '燃爆',
  '更好',
  '理由',
  '不再',
  '生气',
  '绿巨黑',
  '寡激萌',
  '感情',
  '支线',
  '快银',
  '兄妹',
  '同情',
  '牌',
  '台北',
  '日新',
  '威秀',
  '影城'],
 ['奥创', '弱', '爆', '弱', '爆', '弱', '爆'],
 ['第一集',
  '不同',
  '承上启下',
  '阴郁',
  '严肃',
  '不会',
  '好看',
  '本来',
  '喜欢',
  '漫威',
  '电影',
  '场面',
  '更加',
  '宏大',
  '单打',
  '团战',
  '炫技',
  '时刻',
  '镜头感',
  '超赞',
  '故事',
  '平铺',
  '紧凑',
  '冷笑',
  '话',
  '依然',
  '不少',
  '大量',
  '彩蛋',
  '粉丝',
  '菜',
  '结尾',
  '新复联',
  '成立',
  '那一刻',
  '燃爆',
  '不足之处',
  '几大新',
  '角色',
  '没有',
  '展开',
  '感情',
  '戏',
  '生硬',
  '国内',
  '翻译'],
 ['看毕',
  '激动',
  '友人',
  '说',
  '奥创',
  '毁灭',
  '台北',
  '厚',
  '拍了拍',
  '肩膀',
  '没事',
  '反正',
  '买',
  '两份',
  '旅行',
  '保险',
  '惹'],
 ['绝逼', '质疑', '尾灯', '导演', '编剧', '水平'],
 ['睡着', '次', 

In [17]:
ss = [['奥创', '知道', '整容', '韩国'], ['奥创']]



# ['奥创', '知道', '整容', '韩国', '奥创']

[w for s in ss for w in s]

['奥创', '知道', '整容', '韩国', '奥创']

In [18]:
# TODO4: 去除低频词, 去掉词频小于10的单词，并把结果存放在data['comment_processed']里

word_counter = Counter([w for s in data['comment_processed'].values for w in s])


def rm_low_frequency_words(word_list):
    return [w for w in word_list if word_counter[w] >= 10]

data['comment_processed'] = data['comment_processed'].progress_apply(rm_low_frequency_words)

apply: 100%|██████████| 1000/1000 [00:00<00:00, 144820.94it/s]


In [19]:
word_counter.most_common(10)

[('没有', 120),
 ('哐', 107),
 ('电影', 105),
 ('剧情', 100),
 ('没', 98),
 ('感觉', 85),
 ('好看', 81),
 ('第一部', 74),
 ('漫威', 63),
 ('觉得', 62)]

In [20]:
data.head(50)

,Comment,Star,comment_clean,comment_processed
0,连奥创都知道整容要去韩国。,1,连奥创都知道整容要去韩国,"[奥创, 知道]"
1,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,1,一个没有黑暗面的人不值得信任 第二部剥去冗长的铺垫 开场即高潮 一直到结束 会有人觉得只...,"[一个, 没有, 第二部, 冗长, 一直, 会, 觉得, 动作, 蛮, 联盟, 依然, 感情..."
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,0,奥创弱爆了弱爆了弱爆了啊,"[奥创, 弱, 爆, 弱, 爆, 弱, 爆]"
3,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,1,与第一集不同 承上启下 阴郁严肃 但也不会不好看啊 除非本来就不喜欢漫威电影 场面更加宏大 ...,"[好看, 喜欢, 漫威, 电影, 场面, 故事, 话, 依然, 不少, 彩蛋, 角色, 没有..."
4,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,1,看毕 我激动地对友人说 等等奥创要来毁灭台北怎么办厚 她拍了拍我肩膀 没事 反正你买了两份旅...,"[说, 奥创]"
5,绝逼不质疑尾灯的导演和编剧水平,1,绝逼不质疑尾灯的导演和编剧水平,[编剧]
6,avengers1睡着1次 avengers2睡着两次。。。,0,睡着 次 睡着两次,"[睡着, 睡着]"
7,谁再喊我看这种电影我和谁急！实在是接受无能。。。,0,谁再喊我看这种电影我和谁急 实在是接受无能,"[这种, 电影, 实在]"
8,超愉悦以及超满足。在历经了第一阶段比漫画更普世的设定融合之后，发展到#AoU#居然出现了不...,1,超愉悦以及超满足 在历经了第一阶段比漫画更普世的设定融合之后 发展到 居然出现了不少传统...,"[设定, 居然, 出现, 不少, 真的, 超级, 英雄, 漫威, 漫威]"
9,观影过程中，耳边一直有一种突突突突突的声音，我还感慨电影为了让奥创给观众带来紧张感，声音上...,1,观影过程中 耳边一直有一种突突突突突的声音 我还感慨电影为了让奥创给观众带来紧张感 声音上真...,"[观影, 中, 一直, 电影, 奥创, 真是]"


In [21]:
# 排除空样本
data = data[data['comment_processed'].apply(len) > 0]
data.head(10)

,Comment,Star,comment_clean,comment_processed
0,连奥创都知道整容要去韩国。,1,连奥创都知道整容要去韩国,"[奥创, 知道]"
1,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,1,一个没有黑暗面的人不值得信任 第二部剥去冗长的铺垫 开场即高潮 一直到结束 会有人觉得只...,"[一个, 没有, 第二部, 冗长, 一直, 会, 觉得, 动作, 蛮, 联盟, 依然, 感情..."
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,0,奥创弱爆了弱爆了弱爆了啊,"[奥创, 弱, 爆, 弱, 爆, 弱, 爆]"
3,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,1,与第一集不同 承上启下 阴郁严肃 但也不会不好看啊 除非本来就不喜欢漫威电影 场面更加宏大 ...,"[好看, 喜欢, 漫威, 电影, 场面, 故事, 话, 依然, 不少, 彩蛋, 角色, 没有..."
4,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,1,看毕 我激动地对友人说 等等奥创要来毁灭台北怎么办厚 她拍了拍我肩膀 没事 反正你买了两份旅...,"[说, 奥创]"
5,绝逼不质疑尾灯的导演和编剧水平,1,绝逼不质疑尾灯的导演和编剧水平,[编剧]
6,avengers1睡着1次 avengers2睡着两次。。。,0,睡着 次 睡着两次,"[睡着, 睡着]"
7,谁再喊我看这种电影我和谁急！实在是接受无能。。。,0,谁再喊我看这种电影我和谁急 实在是接受无能,"[这种, 电影, 实在]"
8,超愉悦以及超满足。在历经了第一阶段比漫画更普世的设定融合之后，发展到#AoU#居然出现了不...,1,超愉悦以及超满足 在历经了第一阶段比漫画更普世的设定融合之后 发展到 居然出现了不少传统...,"[设定, 居然, 出现, 不少, 真的, 超级, 英雄, 漫威, 漫威]"
9,观影过程中，耳边一直有一种突突突突突的声音，我还感慨电影为了让奥创给观众带来紧张感，声音上...,1,观影过程中 耳边一直有一种突突突突突的声音 我还感慨电影为了让奥创给观众带来紧张感 声音上真...,"[观影, 中, 一直, 电影, 奥创, 真是]"


In [22]:
data['Star'].value_counts()

1    735
0    129
Name: Star, dtype: int64

In [23]:
# 将分词的单词合并为一句话
data['comment_processed_str'] = data['comment_processed'].apply(lambda x: ' '.join(x))
data.head(10)

,Comment,Star,comment_clean,comment_processed,comment_processed_str
0,连奥创都知道整容要去韩国。,1,连奥创都知道整容要去韩国,"[奥创, 知道]",奥创 知道
1,“一个没有黑暗面的人不值得信任。” 第二部剥去冗长的铺垫，开场即高潮、一直到结束，会有人觉...,1,一个没有黑暗面的人不值得信任 第二部剥去冗长的铺垫 开场即高潮 一直到结束 会有人觉得只...,"[一个, 没有, 第二部, 冗长, 一直, 会, 觉得, 动作, 蛮, 联盟, 依然, 感情...",一个 没有 第二部 冗长 一直 会 觉得 动作 蛮 联盟 依然 感情 快银
2,奥创弱爆了弱爆了弱爆了啊！！！！！！,0,奥创弱爆了弱爆了弱爆了啊,"[奥创, 弱, 爆, 弱, 爆, 弱, 爆]",奥创 弱 爆 弱 爆 弱 爆
3,与第一集不同，承上启下，阴郁严肃，但也不会不好看啊，除非本来就不喜欢漫威电影。场面更加宏大...,1,与第一集不同 承上启下 阴郁严肃 但也不会不好看啊 除非本来就不喜欢漫威电影 场面更加宏大 ...,"[好看, 喜欢, 漫威, 电影, 场面, 故事, 话, 依然, 不少, 彩蛋, 角色, 没有...",好看 喜欢 漫威 电影 场面 故事 话 依然 不少 彩蛋 角色 没有 感情 戏 翻译
4,看毕，我激动地对友人说，等等奥创要来毁灭台北怎么办厚，她拍了拍我肩膀，没事，反正你买了两份...,1,看毕 我激动地对友人说 等等奥创要来毁灭台北怎么办厚 她拍了拍我肩膀 没事 反正你买了两份旅...,"[说, 奥创]",说 奥创
5,绝逼不质疑尾灯的导演和编剧水平,1,绝逼不质疑尾灯的导演和编剧水平,[编剧],编剧
6,avengers1睡着1次 avengers2睡着两次。。。,0,睡着 次 睡着两次,"[睡着, 睡着]",睡着 睡着
7,谁再喊我看这种电影我和谁急！实在是接受无能。。。,0,谁再喊我看这种电影我和谁急 实在是接受无能,"[这种, 电影, 实在]",这种 电影 实在
8,超愉悦以及超满足。在历经了第一阶段比漫画更普世的设定融合之后，发展到#AoU#居然出现了不...,1,超愉悦以及超满足 在历经了第一阶段比漫画更普世的设定融合之后 发展到 居然出现了不少传统...,"[设定, 居然, 出现, 不少, 真的, 超级, 英雄, 漫威, 漫威]",设定 居然 出现 不少 真的 超级 英雄 漫威 漫威
9,观影过程中，耳边一直有一种突突突突突的声音，我还感慨电影为了让奥创给观众带来紧张感，声音上...,1,观影过程中 耳边一直有一种突突突突突的声音 我还感慨电影为了让奥创给观众带来紧张感 声音上真...,"[观影, 中, 一直, 电影, 奥创, 真是]",观影 中 一直 电影 奥创 真是


In [24]:
data.shape

(864, 5)

### 2. 把文本分为训练集和测试集
选择语料库中的20%作为测试数据，剩下的作为训练数据

In [25]:
# TODO5: 把数据分为训练集和测试集. comments_train（list)保存用于训练的文本，comments_test(list)保存用于测试的文本。 y_train, y_test是对应的标签（0、1）

from sklearn.model_selection import train_test_split

test_ratio = 0.2

# https://machinelearningmastery.com/train-test-split-for-evaluating-machine-learning-algorithms/
src_training, src_testing = train_test_split(data, test_size=test_ratio, stratify=data['Star'])

comments_train, comments_test = src_training['comment_processed_str'].values, src_testing['comment_processed_str'].values
y_train, y_test = src_training['Star'].values, src_testing['Star'].values

In [26]:
type(comments_train)

numpy.ndarray

In [27]:
comments_train.shape

(691,)

In [28]:
comments_test.shape

(173,)

In [29]:
y_train.shape

(691,)

In [30]:
y_test.shape

(173,)

In [31]:
src_training['Star'].value_counts()

1    588
0    103
Name: Star, dtype: int64

In [32]:
src_testing['Star'].value_counts()

1    147
0     26
Name: Star, dtype: int64

### 3. 把文本转换成向量的形式

在这个部分我们会采用三种不同的方式:
- 使用tf-idf向量
- 使用word2vec
- 使用bert向量

转换成向量之后，我们接着做模型的训练

#### 任务6：把文本转换成tf-idf向量

In [33]:
# TODO6: 把训练文本和测试文本转换成tf-idf向量。使用sklearn的feature_extraction.text.TfidfTransformer模块
#    请留意fit_transform和transform之间的区别。 常见的错误是在训练集和测试集上都使用 fit_transform，需要避免！ 
#    另外，可以留意一下结果是否为稀疏矩阵

# 
# from sklearn.feature_extraction.text import TfidfVectorizer
# vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
# tfidf_train = vectorizer.fit_transform(comments_train)
# tfidf_test = vectorizer.transform(comments_test)

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer

count_vectorizer = CountVectorizer(token_pattern=r"(?u)\b\w+\b")
tfidf_transformer = TfidfTransformer()

word_count_train = count_vectorizer.fit_transform(comments_train)
tfidf_train = tfidf_transformer.fit_transform(word_count_train)

word_count_test = count_vectorizer.transform(comments_test)
tfidf_test = tfidf_transformer.transform(word_count_test)

print(tfidf_train.shape, tfidf_test.shape)

(691, 200) (173, 200)


In [34]:
# TfidfVectorizer = CountVectorizer + TfidfTransformer

from sklearn.feature_extraction.text import TfidfVectorizer
vectorizer = TfidfVectorizer(token_pattern=r"(?u)\b\w+\b")
tfidf_train = vectorizer.fit_transform(comments_train)
tfidf_test = vectorizer.transform(comments_test)
print(tfidf_train.shape, tfidf_test.shape)

# vectorizer.get_feature_names()[:10]

(691, 200) (173, 200)


#### 任务7：把文本转换成word2vec向量

In [35]:
# 由于训练出一个高效的word2vec词向量往往需要非常大的语料库与计算资源，所以我们通常不自己训练Wordvec词向量，而直接使用网上开源的已训练好的词向量。
# data/sgns.zhihu.word是从https://github.com/Embedding/Chinese-Word-Vectors下载到的预训练好的中文词向量文件
# 使用KeyedVectors.load_word2vec_format()函数加载预训练好的词向量文件

# 可以作为词典使用！
model = KeyedVectors.load_word2vec_format('data/sgns.zhihu.word')

In [36]:
#预训练词向量使用举例
model['今天']

array([-3.51068e-01,  2.57389e-01, -1.46752e-01, -4.45400e-03,
       -1.04235e-01,  3.72475e-01, -4.29349e-01, -2.80470e-02,
        1.56651e-01, -1.27600e-01, -1.68833e-01, -2.91350e-02,
        4.57850e-02, -3.53735e-01,  1.61205e-01, -1.82645e-01,
       -1.35340e-02, -2.42591e-01, -1.33356e-01, -1.31012e-01,
       -9.29500e-02, -1.70479e-01, -2.54004e-01, -1.20530e-01,
       -1.33690e-01,  7.84360e-02, -1.46603e-01, -2.77378e-01,
       -1.36723e-01,  9.29070e-02, -4.00197e-01,  2.80726e-01,
       -1.73282e-01,  8.56630e-02,  2.37251e-01,  6.24290e-02,
       -1.57132e-01,  2.15685e-01,  9.54770e-02,  1.09896e-01,
       -2.05394e-01, -3.37900e-03, -2.77480e-02,  8.16580e-02,
        9.65290e-02,  1.23188e-01,  9.55090e-02, -2.31017e-01,
       -8.59590e-02, -2.21634e-01, -1.37885e-01, -1.84790e-01,
       -2.40127e-01, -2.79150e-01, -4.56200e-03,  1.04099e-01,
        3.20523e-01, -6.77270e-02,  1.95719e-01,  4.06145e-01,
       -2.98546e-01, -1.67750e-02,  2.74917e-01, -9.023

In [37]:
model.vocab

{'，': <gensim.models.keyedvectors.Vocab at 0x122127710>,
 '的': <gensim.models.keyedvectors.Vocab at 0x1221275f8>,
 '。': <gensim.models.keyedvectors.Vocab at 0x1221276a0>,
 '了': <gensim.models.keyedvectors.Vocab at 0x122127588>,
 '和': <gensim.models.keyedvectors.Vocab at 0x122127748>,
 '是': <gensim.models.keyedvectors.Vocab at 0x1221276d8>,
 '、': <gensim.models.keyedvectors.Vocab at 0x122127630>,
 '一个': <gensim.models.keyedvectors.Vocab at 0x122127668>,
 '我': <gensim.models.keyedvectors.Vocab at 0x122127780>,
 '有': <gensim.models.keyedvectors.Vocab at 0x122127898>,
 '（': <gensim.models.keyedvectors.Vocab at 0x122127828>,
 '：': <gensim.models.keyedvectors.Vocab at 0x122127860>,
 '不': <gensim.models.keyedvectors.Vocab at 0x1221277b8>,
 '都': <gensim.models.keyedvectors.Vocab at 0x1221277f0>,
 '在': <gensim.models.keyedvectors.Vocab at 0x122127ef0>,
 '上': <gensim.models.keyedvectors.Vocab at 0x1221278d0>,
 '他': <gensim.models.keyedvectors.Vocab at 0x122127eb8>,
 '个': <gensim.models.keyedvect

In [38]:
comments_train[100]

'爆米花 电影'

In [39]:
# TODO7: 对于每个句子，生成句子的向量。具体的做法是：包含在句子中的所有单词的向量做平均。
vocabulary = model.vocab

def word_vec_averaging(words, dim=300):
    """
    Average all words vectors in one sentence.
    :param words: input sentence
    :param dim: 'size' of model
    :return: the averaged word vectors as the vector for the sentence
    """
    vec_mean = np.zeros((dim,), dtype=np.float32)
    word_num = 0
    first_dim_sum = 0
    for word in words:
        print(f'Word is: {word}')
        if word in vocabulary:
            word_num += 1
            vec_mean = np.add(vec_mean, model[word])
            first_dim_sum += model[word][0]
            print(f'in vocab with first dimension: {model[word][0]} and first_dim_sum is: {first_dim_sum}')
        else:
            print('not in vocab')
    if word_num > 0:
        vec_mean = np.divide(vec_mean, word_num)
        print(f'word_num is: {word_num}, first dim average is: {first_dim_sum / word_num}')
    return vec_mean

one_sample = comments_train[100]
word_vec_averaging(one_sample.split())

Word is: 爆米花
in vocab with first dimension: 0.2563000023365021 and first_dim_sum is: 0.2563000023365021
Word is: 电影
in vocab with first dimension: -0.07823099941015244 and first_dim_sum is: 0.17806900292634964
word_num is: 2, first dim average is: 0.08903450146317482


array([ 8.90344977e-02,  3.35854501e-01, -6.14525005e-02,  1.25900004e-03,
        2.69237995e-01,  8.71899910e-03, -2.53995001e-01, -1.55638501e-01,
        3.88228476e-01,  1.40728503e-01, -3.04383993e-01, -2.96440050e-02,
       -1.15949504e-01,  1.90418005e-01, -3.07642490e-01, -1.89981997e-01,
       -2.93444514e-01, -9.57909971e-02,  2.12769993e-02, -3.67630482e-01,
        2.83558011e-01,  2.67385006e-01, -5.60019493e-01, -4.27294970e-02,
       -3.95184979e-02,  4.36163992e-01, -4.41651523e-01,  6.59594983e-02,
       -6.09607995e-01, -1.82741508e-01,  1.46622494e-01,  9.36255008e-02,
       -1.27124995e-01,  6.73779994e-02,  1.27526999e-01, -6.53640032e-02,
        1.91165999e-01,  1.69915497e-01,  5.81149980e-02, -1.15683004e-01,
        3.17643493e-01, -2.68851519e-01,  2.30480492e-01,  1.35015994e-01,
       -1.54837504e-01,  1.88269988e-02, -3.02693009e-01, -2.52121985e-01,
       -2.16813505e-01, -2.92504996e-01,  1.91813007e-01, -1.63973510e-01,
       -4.40684929e-02,  

In [40]:
# TODO7: 对于每个句子，生成句子的向量。具体的做法是：包含在句子中的所有单词的向量做平均。
vocabulary = model.vocab

def word_vec_averaging(words, dim=300):
    """
    Average all words vectors in one sentence.
    :param words: input sentence
    :param dim: 'size' of model
    :return: the averaged word vectors as the vector for the sentence
    """
    vec_mean = np.zeros((dim,), dtype=np.float32)
    word_num = 0
    first_dim_sum = 0
    for word in words:
        if word in vocabulary:
            word_num += 1
            vec_mean = np.add(vec_mean, model[word])
            first_dim_sum += model[word][0]
    if word_num > 0:
        vec_mean = np.divide(vec_mean, word_num)
    return vec_mean

word2vec_train = np.array([word_vec_averaging(s.split()) for s in comments_train])
word2vec_test = np.array([word_vec_averaging(s.split()) for s in comments_test])
print(word2vec_train.shape, word2vec_test.shape)

(691, 300) (173, 300)


In [41]:
word2vec_train[0]

array([-0.17306824,  0.43886498, -0.20718126, -0.08141425,  0.03341125,
        0.09849524, -0.3213545 , -0.14769526, -0.21694   , -0.11872424,
        0.03426275,  0.15320325, -0.12911375, -0.27827102, -0.20194949,
        0.18647   , -0.13373399,  0.0926225 ,  0.035338  ,  0.12461975,
        0.06455475,  0.164865  , -0.00372225, -0.0626615 ,  0.084563  ,
        0.26070774,  0.0098855 , -0.14290625, -0.082787  ,  0.08182075,
       -0.19170174, -0.12172625, -0.11188626,  0.05926025,  0.25832474,
        0.089843  ,  0.2133715 ,  0.23586749,  0.11175825, -0.06057925,
       -0.01089076,  0.0111015 ,  0.024624  , -0.0533405 ,  0.06482724,
        0.051526  , -0.00808199, -0.07853401, -0.2662195 , -0.25124076,
        0.12766176, -0.15248999, -0.35519952, -0.05115625,  0.10924675,
       -0.124053  ,  0.07743075, -0.047606  ,  0.18144001,  0.21708775,
       -0.010286  , -0.08915251,  0.09856025,  0.17224675,  0.0877955 ,
       -0.06926725,  0.1014295 ,  0.4671255 ,  0.404661  , -0.16

#### 任务8：把文本转换成bert向量

In [42]:
# [1] for bert_embedding
# 导入gpu版本的bert embedding预训练的模型。
# 若没有gpu，则ctx可使用其默认值cpu(0)。但使用cpu会使程序运行的时间变得非常慢
# 若之前没有下载过bert embedding预训练的模型，执行此句时会花费一些时间来下载预训练的模型
# ctx = mxnet.gpu()

ctx = mxnet.cpu(0)
embedding = BertEmbedding(model='bert_12_768_12', dataset_name='wiki_cn', ctx=ctx)

# [2] for bert-as-service
# https://github.com/google-research/bert
# https://github.com/hanxiao/bert-as-service
# pip install bert-serving-server  # server
# pip install bert-serving-client  # client, independent of `bert-serving-server`
# bert-serving-start -model_dir /tmp/english_L-12_H-768_A-12/

In [43]:
# [1] for bert_embedding
tokens, token_embeddings = embedding(['今天天气不错'])[0]

# [2] for bert-as-service
# from bert_serving.client import BertClient
# bc = BertClient()
# bc.encode(['First do it', 'then do it right', 'then do it better'])

In [44]:
tokens

['[UNK]', '今', '天', '天', '气', '不', '错', '[UNK]']

In [45]:
token_embeddings

[array([ 1.34265766e-01, -5.34826458e-01, -1.18157375e+00, -6.43333554e-01,
         1.37152457e+00,  1.30425707e-01, -1.44972324e+00,  8.76395822e-01,
        -2.01727796e+00,  1.52168170e-01, -7.82354653e-01, -2.36153394e-01,
         2.03080587e-02,  7.64263868e-01, -5.83726525e-01, -1.34898829e+00,
         1.11576833e-01,  2.41549596e-01,  1.87564790e-01, -5.24836108e-02,
        -2.28334594e+00, -3.72399539e-02,  3.18040907e-01, -3.69692624e-01,
        -2.13828608e-01,  8.61192107e-01, -8.37880254e-01,  1.39965564e-01,
         8.12156320e-01, -1.22752421e-01, -4.32629526e-01,  8.40936184e-01,
         1.11442673e+00,  9.06258941e-01,  6.30802736e-02,  1.26227573e-01,
         4.00351644e-01,  2.04536295e+00, -1.01949906e+00, -5.46246827e-01,
        -9.72957671e-01,  1.00333655e+00, -2.14943796e-01, -2.91988194e-01,
         4.13465887e-01,  9.97146815e-02, -1.26031578e+00, -4.35821936e-02,
        -1.45205474e+00,  3.19183320e-02,  1.99793316e-02,  8.12719631e+00,
         1.0

In [46]:
np.mean(np.array(token_embeddings), axis=0).astype(np.float32)

array([ 3.08819026e-01, -1.55041158e-01, -9.60712284e-02, -1.29176840e-01,
        8.59747112e-01, -1.70599595e-01, -5.90043128e-01,  6.18405342e-01,
       -9.13847864e-01, -6.76962435e-02, -1.50947154e-01, -1.00331020e+00,
        5.76826409e-02,  7.05665410e-01, -1.92537427e-01, -4.00455177e-01,
        7.07005084e-01, -8.66329074e-02,  6.81621373e-01,  5.88591456e-01,
       -1.18335104e+00,  5.18939137e-01, -2.72755744e-03, -1.32210657e-01,
       -1.45531800e-02, -3.79453674e-02, -6.72836423e-01, -5.46709299e-01,
        1.37248427e-01, -1.39747635e-01, -1.92317292e-01,  2.31797665e-01,
        7.72349298e-01,  5.02437353e-01,  9.20175791e-01,  9.26198903e-03,
        1.18876040e+00,  1.05916953e+00, -1.60098600e+00,  8.60936522e-01,
       -3.49327743e-01, -6.44741356e-01,  3.37408036e-02, -7.53357410e-02,
        4.60360259e-01,  7.10076272e-01, -8.32295299e-01, -3.89469475e-01,
       -4.55627441e-01,  3.94381821e-01, -4.34087932e-01,  8.93975639e+00,
       -1.08105771e-01, -

In [47]:
sum([e[0] for e in token_embeddings]) / len(token_embeddings)

0.30881903786212206

In [48]:
def bert_embedding_averaging(sentence):
    """返回sentence bert 句向量"""
    tokens, token_embeddings = embedding([sentence])[0]
    return np.mean(np.array(token_embeddings), axis=0).astype(np.float32)

In [49]:
# TODO8: 跟word2vec一样，计算出训练文本和测试文本的向量，仍然采用单词向量的平均。
bert_train = np.array([bert_embedding_averaging(s) for s in comments_train])
bert_test = np.array([bert_embedding_averaging(s) for s in comments_test])
print (bert_train.shape, bert_test.shape)

(691, 768) (173, 768)


In [50]:
print (tfidf_train.shape, tfidf_test.shape)
print (word2vec_train.shape, word2vec_test.shape)
print (bert_train.shape, bert_test.shape)

(691, 200) (173, 200)
(691, 300) (173, 300)
(691, 768) (173, 768)


### 4. 训练模型以及评估
对如上三种不同的向量表示法，分别训练逻辑回归模型，需要做：
- 搭建模型
- 训练模型（并做交叉验证）
- 输出最好的结果

In [51]:
# 导入逻辑回归的包
from sklearn.linear_model import LogisticRegression

#### 任务9：使用tf-idf，并结合逻辑回归训练模型

In [52]:
# TODO9: 使用tf-idf + 逻辑回归训练模型，需要用gridsearchCV做交叉验证，并选择最好的超参数
clf = LogisticRegression()

from sklearn.model_selection import GridSearchCV

search_grid = {
    'C': [0.01, 1, 10, 100],
    'class_weight': [None, 'balanced']
}

grid_search = GridSearchCV(estimator = clf, 
                           param_grid = search_grid, 
                           cv = 55, 
                           n_jobs=-1, 
                           scoring='accuracy')

grid_result = grid_search.fit(tfidf_train, y_train)


print(f'Best parameters: {grid_result.best_params_}')

Best parameters: {'C': 1, 'class_weight': None}


In [53]:
# TODO9: 使用tf-idf + 逻辑回归训练模型，需要用gridsearchCV做交叉验证，并选择最好的超参数
lr = LogisticRegression(C=1, class_weight=None)
lr.fit(tfidf_train, y_train)
tf_idf_y_pred = lr.predict(tfidf_test)

print('TF-IDF LR test accuracy %s' % metrics.accuracy_score(y_test, tf_idf_y_pred))
#逻辑回归模型在测试集上的F1_Score
print('TF-IDF LR test F1_score %s' % metrics.f1_score(y_test, tf_idf_y_pred,average="macro"))

TF-IDF LR test accuracy 0.8439306358381503
TF-IDF LR test F1_score 0.45768025078369906


#### 任务10：使用word2vec，并结合逻辑回归训练模型

In [54]:
# TODO10: 使用word2vec + 逻辑回归训练模型，需要用gridsearchCV做交叉验证，并选择最好的超参数
clf = LogisticRegression()

from sklearn.model_selection import GridSearchCV

search_grid = {
    'C': [0.01, 1, 10, 100],
    'class_weight': [None, 'balanced']
}

grid_search = GridSearchCV(estimator = clf, 
                           param_grid = search_grid, 
                           cv = 5, 
                           n_jobs=-1, 
                           scoring='accuracy')

grid_result = grid_search.fit(word2vec_train, y_train)

print(f'Best parameters: {grid_result.best_params_}')

Best parameters: {'C': 0.01, 'class_weight': None}


In [55]:
lr = LogisticRegression(C=0.01, class_weight=None)
lr.fit(word2vec_train, y_train)
word2vec_y_pred = lr.predict(word2vec_test)

print('Word2vec LR test accuracy %s' % metrics.accuracy_score(y_test, word2vec_y_pred))
#逻辑回归模型在测试集上的F1_Score
print('Word2vec LR test F1_score %s' % metrics.f1_score(y_test, word2vec_y_pred,average="macro"))

Word2vec LR test accuracy 0.8497109826589595
Word2vec LR test F1_score 0.459375


#### 任务11：使用bert，并结合逻辑回归训练模型

In [56]:
# TODO11: 使用bert + 逻辑回归训练模型，需要用gridsearchCV做交叉验证，并选择最好的超参数

lr = LogisticRegression()
lr.fit(bert_train, y_train)
bert_y_pred = lr.predict(bert_test)

print('Bert LR test accuracy %s' % metrics.accuracy_score(y_test, bert_y_pred))
#逻辑回归模型在测试集上的F1_Score
print('Bert LR test F1_score %s' % metrics.f1_score(y_test, bert_y_pred,average="macro"))

Bert LR test accuracy 0.7572254335260116
Bert LR test F1_score 0.4531908488862131


#### 任务12：对于以上结果请做一下简单的总结，按照1，2，3，4提取几个关键点，包括：
- 结果说明什么问题？
- 接下来如何提高？
    - TFIDF weighted word vectors
    - BERT fine-tuning
    - CNN / RNN, Attention
    - 样本不均衡
        - 数据：采样；增强 (SMOTE, EDA, UDA)
        - 模型
            - 损失函数：Focal loss; [class weight 1](https://www.analyticsvidhya.com/blog/2020/10/improve-class-imbalance-class-weights/); [class weight 2](https://discuss.pytorch.org/t/what-is-the-weight-values-mean-in-torch-nn-crossentropyloss/11455/10)
            - 集成模型：随机森林
            - one-vs-rest

1.
2.
3.
4.
5.
6.